In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

ONLY METABOLOMIC FEATURES

In [19]:
dfmetabolomics = pd.read_excel('C:/Users/jor14/Desktop/Metabolomic Data/metabolomic_features_id_SINO.xlsx')

# Separar la columna 'ID'
ids = dfmetabolomics['ID']  # Guardar los IDs por separado
dfmetabolomics_SIN_ID = dfmetabolomics.drop(columns=['ID'])

# Aplicar la imputación
imp_mean = IterativeImputer(random_state=0)
data = imp_mean.fit_transform(dfmetabolomics_SIN_ID)
df_combined_m_no_id_imputed = pd.DataFrame(data, columns=dfmetabolomics_SIN_ID.columns)

# Añadir de nuevo la columna 'ID' al DataFrame imputado
df_combined_m_imputed = df_combined_m_no_id_imputed.copy()
df_combined_m_imputed['ID'] = ids.values

df_combined_m_imputed.to_excel('C:/Users/jor14/Desktop/Metabolomic Data/metabolomic_nulls.xlsx', index=False)

dfresult = pd.read_excel('C:/Users/jor14/Desktop/Metabolomic Data/result_excel.xlsx')
df_combined_m_imputed = pd.merge(df_combined_m_imputed, dfresult, on='ID', how='outer')

# Mostrar las primeras filas del DataFrame imputado
print(df_combined_m_imputed.head())
# Guardar el resultado en un nuevo archivo Excel
df_combined_m_imputed.to_excel('C:/Users/jor14/Desktop/Metabolomic Data/metabolomic_nulls_NO.xlsx', index=False)

# Eliminar la columna específica
columna_a_eliminar = 'ID'  # Reemplaza con el nombre de la columna que deseas eliminar
if columna_a_eliminar in df_combined_m_imputed.columns:
    dfmetabolomics = df_combined_m_imputed.drop(columns=[columna_a_eliminar])


# Guardar el resultado en un nuevo archivo Excel
dfmetabolomics.to_excel('C:/Users/jor14/Desktop/Metabolomic Data/metabolomic.xlsx', index=False)

print("Archivos 'metabolomic' combinados con éxito.")

         IL6  Ferritin    PCR    LDH    CD4    CD8   DimerD  \
0  44.000000     534.5  14.26  522.0  167.0   71.0   1680.0   
1  28.338293    8836.0  19.97  713.0  122.0   96.0  11320.0   
2   6.440000     449.0   6.44  379.0  351.0  281.0    280.0   
3   7.000000     345.0   6.34  388.0  244.0  117.0    840.0   
4   2.000000     394.6   4.61  237.0  189.0   87.0    740.0   

   Viral_load_baseline  RNA_total_concentration  Glucose  Creatinin    GOT  \
0             0.613046                   479.70    128.0       2.59   44.0   
1             2.468876                   358.05    127.0       0.74  137.0   
2             1.208080                   366.30    116.0       1.18   28.0   
3             1.654463                   348.00    118.0       0.80   29.0   
4             1.055744                   351.20    180.0       1.95   21.0   

    GPT  Leukocytes  Lymphocytes  Platelets    ID  Result  
0  18.0      7210.0        360.0   196000.0   302       0  
1  97.0      5620.0        370.0

ONLY CLINICAL FEATURES

In [7]:
dfclinical = pd.read_excel('C:/Users/jor14/Desktop/Clinical Data/clinical_features_excel_id.xlsx')
dfresult = pd.read_excel('C:/Users/jor14/Desktop/Clinical Data/result_excel.xlsx')

dfclinical.rename(columns={'Gender': 'Sex'}, inplace=True)
dfclinical.Sex=np.where(dfclinical.Sex=='MALE', 1, 0)

# Separar la columna 'ID'
ids = dfclinical['ID']  # Guardar los IDs por separado
dfclinical_SIN_ID = dfclinical.drop(columns=['ID'])

# Aplicar la imputación
imp_mean = IterativeImputer(random_state=0)
data = imp_mean.fit_transform(dfclinical_SIN_ID)
df_combined_c_no_id_imputed = pd.DataFrame(data, columns=dfclinical_SIN_ID.columns)

column_to_exclude = 'PO2_venous'
# Convertir todas las columnas numéricas menos 'C' a enteros (redondeando hacia abajo)
df_combined_c_no_id_imputed[df_combined_c_no_id_imputed.columns.difference([column_to_exclude])] = df_combined_c_no_id_imputed[df_combined_c_no_id_imputed.columns.difference([column_to_exclude])].apply(np.floor).astype(int)


# Añadir de nuevo la columna 'ID' al DataFrame imputado
df_combined_c_imputed = df_combined_c_no_id_imputed.copy()
df_combined_c_imputed['ID'] = ids.values


# Lista de columnas a transformar
columns_to_transform = ['GDS', 'Barthel_scale', 'Pulmonary_affectation']

# Crear dummies/one-hot encoding para las columnas especificadas
df_one_hot = pd.get_dummies(df_combined_c_imputed, columns=columns_to_transform, prefix=columns_to_transform)

df_one_hot.replace({False: 0, True: 1}, inplace=True)

df_one_hot.to_excel('C:/Users/jor14/Desktop/Clinical Data/clinical_ID.xlsx', index=False)

# Combinar los DataFrames en base a la columna 'id'
df_combined_rc = pd.merge(df_one_hot, dfresult, on='ID', how='outer')

# Mostrar las primeras filas del DataFrame imputado
print(df_combined_rc.head())

# Guardar el resultado en un nuevo archivo Excel que contendrá la columna ID para concatenar los otros DF
df_combined_rc.to_excel('C:/Users/jor14/Desktop/Clinical Data/clinical_ID_NO.xlsx', index=False)


# Eliminar la columna específica
columna_a_eliminar = 'ID'  # Reemplaza con el nombre de la columna que deseas eliminar
if columna_a_eliminar in df_combined_rc.columns:
    df_combined_rc = df_combined_rc.drop(columns=[columna_a_eliminar])

df_combined_rc.to_excel('clinical.xlsx', index=False)

print("Archivos 'clinical' combinados con éxito.")

   Age  Sex  Neurological_diseases  Cardiovascular_diseases  \
0   88    0                      0                        1   
1   84    1                      1                        1   
2   87    1                      0                        1   
3   85    0                      1                        1   
4   85    1                      1                        1   

   Respiratory_diseases  Other_diseases  Days_symptoms  Days_hospitalization  \
0                     0               1              7                    13   
1                     1               1              7                    11   
2                     0               1              4                    11   
3                     0               1              4                    43   
4                     0               1              4                    27   

   SAFI  PAFI  ...  Barthel_scale_1  Barthel_scale_2  Barthel_scale_3  \
0   182   332  ...                0                0               

RADIOMIC, CLINICAL AND METABOLOMIC

In [ ]:
# Leer los archivos Excel
dfmetabolomics = pd.read_excel('metabolomic_ID.xlsx')
dfclinical = pd.read_excel('clinical_ID.xlsx')
dfradiomics = pd.read_excel('radiomic_ID.xlsx')

# Combinar los DataFrames en base a la columna 'id'
df_combined_mc = pd.merge(dfmetabolomics, dfclinical, on='ID', how='outer')
df_combined_all = pd.merge(df_combined_mc, dfradiomics, on='ID', how='outer')

# Eliminar la columna específica
columna_a_eliminar = 'ID'  # Reemplaza con el nombre de la columna que deseas eliminar
if columna_a_eliminar in df_combined_mc.columns:
    df_combined_mc = df_combined_mc.drop(columns=[columna_a_eliminar])

if columna_a_eliminar in df_combined_all.columns:
    df_combined_all = df_combined_all.drop(columns=[columna_a_eliminar])

# Guardar el resultado en un nuevo archivo Excel
#df_combined_mc.to_excel('metabolomic&clinical.xlsx', index=False)
df_combined_all.to_excel('radiomics&metabolomic&clinical.xlsx', index=False)

print("Archivos todos combinados con éxito.")

Archivos todos combinados con éxito.
